In [3]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import norm
import multiprocessing

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import sys
sys.path.append(".")
from src.dc_smc import dc_smc 
from src.model import Model
from src.utils import resample

# Linear Gaussian HMM

In [85]:
#! matrix multiplication is wrong!
class LinearGaussian:
    def __init__(self, sigma0, A, B, C, D):
        self.sigma0 = sigma0
        self.mu0 = np.zeros((1, 1))
        self.A = A
        self.B = B
        self.C = C
        self.D = D
    def sample(self, nsamples):
        """sample x_i for i=1:(n+1) and y_i for i=1:n
        """
        x_vec = np.zeros((nsamples+1, 1))
        y_vec = np.zeros((nsamples, 1))
        
        # sample x0 from prior
        x_vec[0, :] = tfd.Normal(loc=self.mu0, scale=self.sigma0).sample(1)
        # sample iteratively
        for i in range(nsamples):
            x_new, y_new = self.sample_once(x_vec[i, :], i)
            x_vec[i+1, :] = x_new
            y_vec[i, :] = y_new
        
        return x_vec, y_vec
    
    def update_sample(self, x_vec, y_vec, n=1):
        """sample n more data
        """
        nsamples, dim = y_vec.shape
        x_vec = np.append(x_vec, np.zeros((n, dim)), axis=0)
        y_vec = np.append(y_vec, np.zeros((n, dim)), axis=0)
        for i in range(n):
            x_new, y_new = self.sample_once(x_vec[i+nsamples, :], i)
            x_vec[i+1+nsamples, :] = x_new
            y_vec[i+nsamples, :] = y_new
        return x_vec, y_vec
            
    def sample_once(self, xt, t):
        x_new = tf.math.multiply(self.A, xt) + \
            tf.math.multiply(self.B, tfd.Normal(loc=0, scale=1).sample(1)).numpy()
        y_new = tf.math.multiply(self.C, x_new) + \
            tf.math.multiply(self.D, tfd.Normal(loc=0, scale=1).sample(1)).numpy()
        return x_new, y_new
            
    def log_gamma(self, x_vec, y_vec):
        """unnormalized log posterior
        """
        log_gamma = 0
        nsamples = x_vec.shape[0]
        
        log_lik = tfd.MultivariateNormalFullCovariance(
            loc=tf.math.multiply(self.C, x_vec[1:, :]), 
            covariance_matrix=self.D @ self.D.T
        ).log_prob(y_vec)
        
        log_p = tfd.Normal(loc=self.mu0, scale=self.sigma0).log_prob(x_vec[0, :])
        for i in range(1, nsamples):
            log_p += tfd.MultivariateNormalFullCovariance(
                loc=tf.math.multiply(self.A, x_vec[i-1, :]), 
                covariance_matrix=self.B @ self.B.T
            ).log_prob(x_vec[i, :])
        return log_p + tf.reduce_sum(log_lik)

In [86]:
sigma0 = np.array([[1.]])
A = np.array([[0.8]])
B = np.ones((1, 1))
C = np.ones((1, 1))
D = np.ones((1, 1))

linear_hmm = LinearGaussian(sigma0, A, B, C, D)
x_vec, y_vec = linear_hmm.sample(5)
x_vec, y_vec

(array([[-0.35295168],
        [-0.61807435],
        [ 0.54944054],
        [ 0.97703505],
        [ 0.94505231],
        [ 2.60163732]]),
 array([[0.20130504],
        [0.77761779],
        [1.16177938],
        [1.11780159],
        [3.4207777 ]]))

In [87]:
linear_hmm.update_sample(x_vec, y_vec, )

(array([[-0.35295168],
        [-0.61807435],
        [ 0.54944054],
        [ 0.97703505],
        [ 0.94505231],
        [ 2.60163732],
        [ 1.16896183]]),
 array([[0.20130504],
        [0.77761779],
        [1.16177938],
        [1.11780159],
        [3.4207777 ],
        [1.37159199]]))

In [88]:
linear_hmm.log_gamma(x_vec, y_vec)

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-13.36194099]])>

# Non-linear Gaussian HMM

In [89]:
#! matrix multiplication is wrong!
class NonLinearGaussian(LinearGaussian):
    def __init__(self, sigma0, sigma_v, sigma_w):
        self.sigma0 = sigma0
        self.mu0 = np.zeros((1, 1))
        self.sigma_v = sigma_v
        self.sigma_w = sigma_w
            
    def _latent_mean(self, xt, t):
        return 0.5 * xt + 25 * xt / (1 + xt**2) + 8 * np.cos(1.2 * t)
    
    def sample_once(self, xt, t):
        x_new = self._latent_mean(xt, t) + \
            tfd.Normal(loc=0, scale=self.sigma_v).sample(1).numpy()
        y_new = 1/20 * x_new**2 + \
            tfd.Normal(loc=0, scale=self.sigma_w).sample(1).numpy()
        return x_new, y_new
            
    def log_gamma(self, x_vec, y_vec):
        """unnormalized log posterior
        """
        log_gamma = 0
        nsamples = x_vec.shape[0]
        
        log_lik = tfd.Normal(
            loc=1/20 * x_vec[1:, :]**2, 
            scale=self.sigma_w
        ).log_prob(y_vec)

        log_p = tfd.Normal(loc=self.mu0, scale=self.sigma0).log_prob(x_vec[0, :])
        for i in range(1, nsamples):
            log_p += tfd.Normal(
                loc=self._latent_mean(x_vec[i-1, :], i), 
                scale=self.sigma_v
            ).log_prob(x_vec[i, :])
        return log_p + tf.reduce_sum(log_lik)

In [91]:
sigma0 = np.array([[1.]])
sigma_v = np.array([[np.sqrt(10)]])
sigma_w = np.ones((1, 1))

nonlinear_hmm = NonLinearGaussian(sigma0, sigma_v, sigma_w)
x_vec2, y_vec2 = nonlinear_hmm.sample(5)
x_vec2, y_vec2

(array([[ 0.90432218],
        [26.43298512],
        [18.74791913],
        [ 0.58583726],
        [ 2.45710654],
        [ 8.00773997]]),
 array([[37.39417966],
        [17.00759914],
        [-1.21182593],
        [ 0.79095997],
        [ 4.98997146]]))

In [92]:
nonlinear_hmm.update_sample(x_vec2, y_vec2, )

(array([[ 0.90432218],
        [26.43298512],
        [18.74791913],
        [ 0.58583726],
        [ 2.45710654],
        [ 8.00773997],
        [17.84976942]]),
 array([[37.39417966],
        [17.00759914],
        [-1.21182593],
        [ 0.79095997],
        [ 4.98997146],
        [15.81851405]]))

In [93]:
nonlinear_hmm.log_gamma(x_vec2, y_vec2)

(5, 1)


<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-42.61334452]])>

# Hierarchical model

In [94]:
def hierarchical_targets(y, sigma0, eta, tau):
    def log_gamma_leaf(x):
        log_lik = tfd.Normal(loc=x, scale=eta).log_prob(y)
        log_p = tfd.Normal(loc=mu0, scale=sigma0).log_prob(x[:, 0])
        log_p += tfd.Normal(loc=x[:, 0], scale=tau).log_prob(x[:, 1:])
        return log_p
    
    def sample_x(nsamples):
        x_vec = np.zeros((nsapmles, 5))
        x_vec[:, 0] = tfd.Normal(loc=mu0, scale=sigma0).sample(nsamples)
        x_vec[:, 1:] = tfd.Normal(loc=x_vec[:, 0], scale=tau).sample((nsamples, x_vec.shape[1]-1))
        return x_vec
    
    def sample(nsamples, x_vec):
        y_vec = tfd.Normal(loc=x_vec[:, 1:], scale=eta).sample(nsamples)
        return y_vec
    